In [1]:
import pandas as pd
import numpy as np

## Input

In [2]:
allinfo_file = "final/correct-mid/Canomad-1-BuH_S231_L001.assembled-ATGCATGC-IGH_HUMAN-all_info.csv"
v_file = "Canomad-1-BuH_S231_L001.assembled-ATGCATGC-IGHV_human-e-clean.sam.mut.txt"
j_file = "Canomad-1-BuH_S231_L001.assembled-ATGCATGC-IGHV_human-e-clean.sam.mut.txt"

## Output

In [3]:
path = allinfo_file.split("/")
mydir = path[:-1]
myfile = path[-1]
(sample, rest) = myfile.split("_L001.assembled-")
mid = rest.split("-")[0]
outfile = "/".join(mydir) + "/" + sample + "-" + mid + "-clones-mut-sites" + ".csv"
print(outfile)

final/correct-mid/Canomad-1-BuH_S231-ATGCATGC-clones-mut-sites.csv


## Read files

In [4]:
allinfo = pd.read_csv(allinfo_file, sep='\t')
print("allinfo entries:", len(allinfo))
allinfo.head()

allinfo entries: 39845


,acc,beforeMID,MID,afterMID,readingframe,cdr3pep,cdr3nuc,cdr3_qual_min,cdr3_qual_max,cdr3_qual_avg,...,V_main,acc:1,nr_v_mains,nr_v_subs,nr_v_alleles,nr_j_subs,nr_j_alleles,acc:2,readingframe:1,nr_sites
0,M04659:194:000000000-BTD9H:1:1101:10045:9521,AATGGATAT,ATGCATGC,CTTAC,5,CARAVTLRDAFDIWGQGTMVT,TGTGCGAGAGCCGTTACGTTGCGCGATGCTTTTGATATCTGGGGCC...,38,40,39.9,...,IGHV1,M04659:194:000000000-BTD9H:1:1101:10045:9521,1,1,1,1,1,None,None,None
1,M04659:194:000000000-BTD9H:1:1101:10089:6210,GGTCGTTTG,ATGCATGC,CTTAC,4,CAREKNYYDFWSGYYTEDVSTYYYYGMDVWGQGTTVT,TGTGCGAGAGAAAAGAATTATTACGATTTTTGGAGTGGTTATTATA...,38,40,39.3,...,IGHV1,M04659:194:000000000-BTD9H:1:1101:10089:6210,1,1,1,1,1,M04659:194:000000000-BTD9H:1:1101:10089:6210,4,1
2,M04659:194:000000000-BTD9H:1:1101:10092:23161,GAATTTAAT,ATGCATGC,CTTAC,4,CARAVATVVRNEHWGQGTLVT,TGTGCGAGAGCAGTGGCTACGGTGGTAAGAAACGAGCACTGGGGCC...,32,40,39.3,...,IGHV1,M04659:194:000000000-BTD9H:1:1101:10092:23161,1,1,1,1,1,None,None,None
3,M04659:194:000000000-BTD9H:1:1101:10092:2562,TAGCACCCA,ATGCATGC,CTTAC,5,CARVRSGSYWGNYYYYYGMDVWGQGTTVT,TGTGCAAGAGTTCGGAGTGGGAGCTACTGGGGGAATTACTACTACT...,23,40,39.0,...,IGHV6,M04659:194:000000000-BTD9H:1:1101:10092:2562,1,1,1,1,1,None,None,None
4,M04659:194:000000000-BTD9H:1:1101:10099:4245,AAAGTGGAA,ATGCATGC,CTTAC,4,CAKDIGVAAAGAYGMDVWGQGTTVT,TGTGCAAAAGATATCGGTGTCGCAGCAGCAGGGGCCTACGGTATGG...,33,40,39.5,...,IGHV3,M04659:194:000000000-BTD9H:1:1101:10099:4245,1,1,1,1,1,None,None,None


In [5]:
# Check if acc in allinfo is unique. ANSWER: no
allinfo['acc'].nunique()

39366

In [6]:
# Check properties of accs in allinfo that are not unique
count_acc_allinfo = allinfo.groupby('acc').agg(['nunique'])
col = 'V_sub'
count_acc_allinfo.sort_values((col,'nunique'), ascending=False)[col].head()

,nunique
acc,
M04659:194:000000000-BTD9H:1:1101:4637:20933,3
M04659:194:000000000-BTD9H:1:2113:21143:7263,3
M04659:194:000000000-BTD9H:1:1113:12371:16143,3
M04659:194:000000000-BTD9H:1:2112:19755:10557,3
M04659:194:000000000-BTD9H:1:1108:16720:14558,3


In [7]:
# Replace 'None' with 0 for the nr_sites column
allinfo['nr_sites'] = allinfo['nr_sites'].replace('None', 0).apply(int)

In [8]:
v = pd.read_csv(v_file, sep=' ')
print("v entries:", len(v))
v.head()

v entries: 48091


,acc,cigar,start.pos,end.pos,mut.count,mut.frac,align.length,align.seq
0,M04659:194:000000000-BTD9H:1:1101:18136:1916,53S193M115S,53,246,10,0.051813,193,===========================C==================...
1,M04659:194:000000000-BTD9H:1:1101:8309:1960,5S235M116S,5,240,2,0.008511,235,===========================================C==...
2,M04659:194:000000000-BTD9H:1:1101:16881:2093,5S236M136S,5,241,3,0.012712,236,============G=================================...
3,M04659:194:000000000-BTD9H:1:1101:21546:2115,249M112S,0,249,16,0.064257,249,====================C===C===CT================...
4,M04659:194:000000000-BTD9H:1:1101:15319:2143,238M144S,0,238,15,0.063025,238,====================A==============C==========...


In [9]:
# Check if acc in v is unique. ANSWER: nope
v['acc'].nunique()

47486

In [10]:
j = pd.read_csv(j_file, sep=' ')
print("j entries:", len(j))
j.head()

j entries: 48091


,acc,cigar,start.pos,end.pos,mut.count,mut.frac,align.length,align.seq
0,M04659:194:000000000-BTD9H:1:1101:18136:1916,53S193M115S,53,246,10,0.051813,193,===========================C==================...
1,M04659:194:000000000-BTD9H:1:1101:8309:1960,5S235M116S,5,240,2,0.008511,235,===========================================C==...
2,M04659:194:000000000-BTD9H:1:1101:16881:2093,5S236M136S,5,241,3,0.012712,236,============G=================================...
3,M04659:194:000000000-BTD9H:1:1101:21546:2115,249M112S,0,249,16,0.064257,249,====================C===C===CT================...
4,M04659:194:000000000-BTD9H:1:1101:15319:2143,238M144S,0,238,15,0.063025,238,====================A==============C==========...


In [11]:
# Check if acc in j is unique. ANSWER: nope
j['acc'].nunique()

47486

## Combine files

In [13]:
df = pd.merge(allinfo, v, how='left', left_on='acc', right_on='acc')
df = pd.merge(df, j, how='left', left_on='acc', right_on='acc')
print("df merged enties:", len(df))
df.head()

df merged enties: 42869


,acc,beforeMID,MID,afterMID,readingframe,cdr3pep,cdr3nuc,cdr3_qual_min,cdr3_qual_max,cdr3_qual_avg,...,mut.frac_x,align.length_x,align.seq_x,cigar_y,start.pos_y,end.pos_y,mut.count_y,mut.frac_y,align.length_y,align.seq_y
0,M04659:194:000000000-BTD9H:1:1101:10045:9521,AATGGATAT,ATGCATGC,CTTAC,5,CARAVTLRDAFDIWGQGTMVT,TGTGCGAGAGCCGTTACGTTGCGCGATGCTTTTGATATCTGGGGCC...,38,40,39.9,...,0.004167,240.0,====T=========================================...,240M110S,0.0,240.0,1.0,0.004167,240.0,====T=========================================...
1,M04659:194:000000000-BTD9H:1:1101:10089:6210,GGTCGTTTG,ATGCATGC,CTTAC,4,CAREKNYYDFWSGYYTEDVSTYYYYGMDVWGQGTTVT,TGTGCGAGAGAAAAGAATTATTACGATTTTTGGAGTGGTTATTATA...,38,40,39.3,...,0.034146,205.0,========================================T=====...,35S205M157S,35.0,240.0,7.0,0.034146,205.0,========================================T=====...
2,M04659:194:000000000-BTD9H:1:1101:10092:23161,GAATTTAAT,ATGCATGC,CTTAC,4,CARAVATVVRNEHWGQGTLVT,TGTGCGAGAGCAGTGGCTACGGTGGTAAGAAACGAGCACTGGGGCC...,32,40,39.3,...,0.041841,239.0,=A=======T====G===========G===================...,239M110S,0.0,239.0,10.0,0.041841,239.0,=A=======T====G===========G===================...
3,M04659:194:000000000-BTD9H:1:1101:10092:2562,TAGCACCCA,ATGCATGC,CTTAC,5,CARVRSGSYWGNYYYYYGMDVWGQGTTVT,TGTGCAAGAGTTCGGAGTGGGAGCTACTGGGGGAATTACTACTACT...,23,40,39.0,...,0.000000,252.0,==============================================...,252M134S,0.0,252.0,0.0,0.000000,252.0,==============================================...
4,M04659:194:000000000-BTD9H:1:1101:10099:4245,AAAGTGGAA,ATGCATGC,CTTAC,4,CAKDIGVAAAGAYGMDVWGQGTTVT,TGTGCAAAAGATATCGGTGTCGCAGCAGCAGGGGCCTACGGTATGG...,33,40,39.5,...,0.000000,242.0,==============================================...,242M119S,0.0,242.0,0.0,0.000000,242.0,==============================================...


## Filter data

In [14]:
df = df.loc[(df['cdr3_qual_min'] >= 30) & (df['V_sub'] != 'None') & (df['J_sub'] != 'None') & ((df['V_flag'] == '0') | (df['V_flag'] == '16')) & ((df['J_flag'] == '0') | (df['J_flag'] == '16'))]
print("df filtered entries:", len(df))
df.head()

df filtered entries: 27978


,acc,beforeMID,MID,afterMID,readingframe,cdr3pep,cdr3nuc,cdr3_qual_min,cdr3_qual_max,cdr3_qual_avg,...,mut.frac_x,align.length_x,align.seq_x,cigar_y,start.pos_y,end.pos_y,mut.count_y,mut.frac_y,align.length_y,align.seq_y
0,M04659:194:000000000-BTD9H:1:1101:10045:9521,AATGGATAT,ATGCATGC,CTTAC,5,CARAVTLRDAFDIWGQGTMVT,TGTGCGAGAGCCGTTACGTTGCGCGATGCTTTTGATATCTGGGGCC...,38,40,39.9,...,0.004167,240.0,====T=========================================...,240M110S,0.0,240.0,1.0,0.004167,240.0,====T=========================================...
1,M04659:194:000000000-BTD9H:1:1101:10089:6210,GGTCGTTTG,ATGCATGC,CTTAC,4,CAREKNYYDFWSGYYTEDVSTYYYYGMDVWGQGTTVT,TGTGCGAGAGAAAAGAATTATTACGATTTTTGGAGTGGTTATTATA...,38,40,39.3,...,0.034146,205.0,========================================T=====...,35S205M157S,35.0,240.0,7.0,0.034146,205.0,========================================T=====...
2,M04659:194:000000000-BTD9H:1:1101:10092:23161,GAATTTAAT,ATGCATGC,CTTAC,4,CARAVATVVRNEHWGQGTLVT,TGTGCGAGAGCAGTGGCTACGGTGGTAAGAAACGAGCACTGGGGCC...,32,40,39.3,...,0.041841,239.0,=A=======T====G===========G===================...,239M110S,0.0,239.0,10.0,0.041841,239.0,=A=======T====G===========G===================...
4,M04659:194:000000000-BTD9H:1:1101:10099:4245,AAAGTGGAA,ATGCATGC,CTTAC,4,CAKDIGVAAAGAYGMDVWGQGTTVT,TGTGCAAAAGATATCGGTGTCGCAGCAGCAGGGGCCTACGGTATGG...,33,40,39.5,...,0.000000,242.0,==============================================...,242M119S,0.0,242.0,0.0,0.000000,242.0,==============================================...
5,M04659:194:000000000-BTD9H:1:1101:10151:4067,TAGACTTTG,ATGCATGC,CTTAC,4,CARASYDFYLDYWGQGTLVT,TGTGCGAGAGCAAGCTACGATTTTTACCTTGACTACTGGGGCCAGG...,38,40,40.0,...,0.004132,242.0,==============================================...,242M106S,0.0,242.0,1.0,0.004132,242.0,==============================================...


In [15]:
# acc in df unique? ANSWER: still not unique
df['acc'].nunique()

27012

In [16]:
# Check why accessions are not unique
tmp = df.groupby('acc').agg('nunique')
tmp.apply(sum)

acc                 27012
beforeMID           27012
MID                 27012
afterMID            27012
readingframe        27012
cdr3pep             27012
cdr3nuc             27012
cdr3_qual_min       27012
cdr3_qual_max       27012
cdr3_qual_avg       27012
cdr3_qual           27012
nt_start            27012
nt_end              27012
seq_length          27012
V_flag              27012
V_gene              27012
J_flag              27012
J_gene              27012
readingframe_seq    27012
seq                 27012
pep                 27012
qual                27012
V_sub               27012
J_sub               27012
V_main              27012
acc:1               27012
nr_v_mains          27012
nr_v_subs           27012
nr_v_alleles        27012
nr_j_subs           27012
nr_j_alleles        27012
acc:2               27012
readingframe:1      27012
nr_sites            27012
cigar_x             27328
start.pos_x         27279
end.pos_x           27320
mut.count_x         27298
mut.frac_x  

## Group data per clone (CDR3pep)

In [17]:
df.columns

Index(['acc', 'beforeMID', 'MID', 'afterMID', 'readingframe', 'cdr3pep',
       'cdr3nuc', 'cdr3_qual_min', 'cdr3_qual_max', 'cdr3_qual_avg',
       'cdr3_qual', 'nt_start', 'nt_end', 'seq_length', 'V_flag', 'V_gene',
       'J_flag', 'J_gene', 'readingframe_seq', 'seq', 'pep', 'qual', 'V_sub',
       'J_sub', 'V_main', 'acc:1', 'nr_v_mains', 'nr_v_subs', 'nr_v_alleles',
       'nr_j_subs', 'nr_j_alleles', 'acc:2', 'readingframe:1', 'nr_sites',
       'cigar_x', 'start.pos_x', 'end.pos_x', 'mut.count_x', 'mut.frac_x',
       'align.length_x', 'align.seq_x', 'cigar_y', 'start.pos_y', 'end.pos_y',
       'mut.count_y', 'mut.frac_y', 'align.length_y', 'align.seq_y'],
      dtype='object')

In [18]:
clones = df.groupby('cdr3pep').agg({'acc': 'nunique', 'beforeMID': 'nunique', 'mut.count_x': [sum, np.mean], 'mut.frac_x': [sum, np.mean], 'mut.count_y': [sum, np.mean], 'mut.frac_y': [sum, np.mean], 'nr_sites': [sum, np.mean]})
clones = clones.sort_values(by=('acc','nunique'), ascending=False)
print("Clones entries:", len(clones))
clones.head()

Clones entries: 15466


acc beforeMID mut.count_x             \
                               nunique   nunique         sum       mean   
cdr3pep                                                                   
CAKDQSGYYTRHTGMDVWGQGTTVT          515       424      5725.0  10.801887   
CARAFVTLAYNDYWGQGTLVT              295       260      3903.0  13.097315   
CARDRGPEGSGSYYNPSYGMDVWGRGTTVT     161       134      2517.0  15.347561   
CARDRTPRYYDSSGQGFDYWGQGTLVT        124        95       455.0   3.582677   
CAKDQSGYHTRHTGMDVWGQGTTVT          104        83      1312.0  12.615385   

                               mut.frac_x           mut.count_y             \
                                      sum      mean         sum       mean   
cdr3pep                                                                      
CAKDQSGYYTRHTGMDVWGQGTTVT       25.514192  0.048140      5725.0  10.801887   
CARAFVTLAYNDYWGQGTLVT           16.365046  0.054916      3903.0  13.097315   
CARDRGPEGSGSYYNPSYGMDVWGRGTTVT  10.597098  0.064616      2517.0  15.347561   
CARDRTPRYYDSSGQGFDYWGQGTLVT      2.025808  0.015951       455.0   3.582677   
CAKDQSGYHTRHTGMDVWGQGTTVT        5.748816  0.055277      1312.0  12.615385   

                               mut.frac_y           nr_sites            
                                      sum      mean      sum      mean  
cdr3pep                                                                 
CAKDQSGYYTRHTGMDVWGQGTTVT       25.514192  0.048140       11  0.020755  
CARAFVTLAYNDYWGQGTLVT           16.365046  0.054916      279  0.936242  
CARDRGPEGSGSYYNPSYGMDVWGRGTTVT  10.597098  0.064616        9  0.054878  
CARDRTPRYYDSSGQGFDYWGQGTLVT      2.025808  0.015951        0  0.000000  
CAKDQSGYHTRHTGMDVWGQGTTVT        5.748816  0.055277        1  0.009615

In [19]:
clones.columns = ['.'.join(col).strip() for col in clones.columns.values]
clones.head()

,acc.nunique,beforeMID.nunique,mut.count_x.sum,mut.count_x.mean,mut.frac_x.sum,mut.frac_x.mean,mut.count_y.sum,mut.count_y.mean,mut.frac_y.sum,mut.frac_y.mean,nr_sites.sum,nr_sites.mean
cdr3pep,,,,,,,,,,,,
CAKDQSGYYTRHTGMDVWGQGTTVT,515,424,5725.0,10.801887,25.514192,0.048140,5725.0,10.801887,25.514192,0.048140,11,0.020755
CARAFVTLAYNDYWGQGTLVT,295,260,3903.0,13.097315,16.365046,0.054916,3903.0,13.097315,16.365046,0.054916,279,0.936242
CARDRGPEGSGSYYNPSYGMDVWGRGTTVT,161,134,2517.0,15.347561,10.597098,0.064616,2517.0,15.347561,10.597098,0.064616,9,0.054878
CARDRTPRYYDSSGQGFDYWGQGTLVT,124,95,455.0,3.582677,2.025808,0.015951,455.0,3.582677,2.025808,0.015951,0,0.000000
CAKDQSGYHTRHTGMDVWGQGTTVT,104,83,1312.0,12.615385,5.748816,0.055277,1312.0,12.615385,5.748816,0.055277,1,0.009615


In [20]:
clones.to_csv(outfile, sep='\t')
print("Wrote", outfile, "to disk")

Wrote final/correct-mid/Canomad-1-BuH_S231-ATGCATGC-clones-mut-sites.csv to disk
